#**Parte 1-** Base de dados

##**1.1 -** Leitura dos dados

df audioset multilabels

In [2]:
#Clonando e acessando a pasta do dataset
!apt-get install git
!git clone https://github.com/gcolussi11/Biochallenge

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.11).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
fatal: destination path 'Biochallenge' already exists and is not an empty directory.


In [ ]:
import json
import pandas as pd

In [ ]:
#Filtrar o dicionário já removendo as labels que não são desejadas
#Apesar de estar escrito negadas, são as que a gente pode quer (eu errei o nome)
with open('/content/Biochallenge/dic_labels_negadas2.json', 'r') as f:
  labels_queridas = json.load(f)

print(labels_queridas)
print(len(labels_queridas))
#print(type(labels_queridas))

{'/m/0dgw9r': 'Human sounds', '/m/09l8g': 'Human voice', '/m/09x0r': 'Speech', '/m/0ytgt': 'kid speaking', '/m/01h8n0': 'Conversation', '/m/07p6fty': 'Shout', '/m/07sr1lc': 'Yell', '/m/04gy_2': 'Battle cry', '/t/dd00135': 'Children shouting', '/m/03qc9zr': 'Screaming', '/m/01j3sz': 'Laughter', '/t/dd00001': 'Baby laughter', '/m/07sq110': 'Belly laugh', '/t/dd00002': 'infant cry', '/m/015lz1': 'Singing', '/m/01j423': 'Yawn', '/m/01w250': 'Whistling', '/m/079vc8': 'Wolf-whistling', '/m/09hlz4': 'Respiratory sounds', '/m/0lyf6': 'Breathing', '/m/01d3sd': 'Snoring', '/m/01hsr_': 'Sneeze', '/m/0bpl036': 'Human locomotion', '/m/07pbtc8': 'footsteps', '/m/0160x5': 'Digestive', '/m/0l15bq': 'Clapping', '/m/04xp5v': 'Otoacoustic emission', '/m/0pv6y': 'ringing in the ears', '/t/dd00012': 'Human group actions', '/m/053hz1': 'Cheering', '/m/028ght': 'Applause', '/m/07rkbfh': 'Chatter', '/m/03qtwd': 'Crowd', '/m/04v5dt': 'Booing', '/t/dd00013': 'Children playing', '/m/0jbk': 'Animal', '/m/068hy': 

In [ ]:
column_names = ['YTID', 'start_seconds', 'end_seconds', 'positive_labels']

#Ler o arquivo e criar um df
df_audioset = pd.read_csv('/content/Biochallenge/balanced_train_segments.csv', skiprows=3, names = column_names, quotechar='"', skipinitialspace=True)
df_audioset.head()

,YTID,start_seconds,end_seconds,positive_labels
0,--PJHxphWEs,30.0,40.0,"/m/09x0r,/t/dd00088"
1,--ZhevVpy1s,50.0,60.0,/m/012xff
2,--aE2O5G5WE,0.0,10.0,"/m/03fwl,/m/04rlf,/m/09x0r"
3,--aO5cdqSAg,30.0,40.0,"/t/dd00003,/t/dd00005"
4,--aaILOrkII,200.0,210.0,"/m/032s66,/m/073cg4"


In [ ]:
# Filtrando as linhas que contêm a label específica
label_to_find = '/m/0dgw9r'
filtered_df = df_audioset[df_audioset['positive_labels'].str.contains(label_to_find)]

# Mostrando o DataFrame filtrado
filtered_df

,YTID,start_seconds,end_seconds,positive_labels


##**1.2 -** Preparação da base

Transformar o df em apenas uma label por linha

In [ ]:
#Dividindo as labels em listas
df_audioset['positive_labels'] = df_audioset['positive_labels'].str.split(',')

In [ ]:
# prompt: procurar as linhas com 'Human voice'

df_human_voice = df_audioset[df_audioset['positive_labels'] == '/m/0dgw9r']
print(df_human_voice.head())


Empty DataFrame
Columns: [YTID, start_seconds, end_seconds, positive_labels]
Index: []


In [ ]:
#"Explodindo essas listas em mais linhas"
df_expandido = df_audioset.explode('positive_labels')

#Removendo linhas com positive labels vazios
df_expandido = df_expandido[df_expandido['positive_labels'] != '']
df_expandido = df_expandido.reset_index(drop=True)

#print(df_expandido.head())
print(len(df_expandido))

52882


Remover as labels indesejadas e adicionar uma coluna com os labels descritivos

In [ ]:
print(labels_queridas)

{'/m/0dgw9r': 'Human sounds', '/m/09l8g': 'Human voice', '/m/09x0r': 'Speech', '/m/0ytgt': 'kid speaking', '/m/01h8n0': 'Conversation', '/m/07p6fty': 'Shout', '/m/07sr1lc': 'Yell', '/m/04gy_2': 'Battle cry', '/t/dd00135': 'Children shouting', '/m/03qc9zr': 'Screaming', '/m/01j3sz': 'Laughter', '/t/dd00001': 'Baby laughter', '/m/07sq110': 'Belly laugh', '/t/dd00002': 'infant cry', '/m/015lz1': 'Singing', '/m/01j423': 'Yawn', '/m/01w250': 'Whistling', '/m/079vc8': 'Wolf-whistling', '/m/09hlz4': 'Respiratory sounds', '/m/0lyf6': 'Breathing', '/m/01d3sd': 'Snoring', '/m/01hsr_': 'Sneeze', '/m/0bpl036': 'Human locomotion', '/m/07pbtc8': 'footsteps', '/m/0160x5': 'Digestive', '/m/0l15bq': 'Clapping', '/m/04xp5v': 'Otoacoustic emission', '/m/0pv6y': 'ringing in the ears', '/t/dd00012': 'Human group actions', '/m/053hz1': 'Cheering', '/m/028ght': 'Applause', '/m/07rkbfh': 'Chatter', '/m/03qtwd': 'Crowd', '/m/04v5dt': 'Booing', '/t/dd00013': 'Children playing', '/m/0jbk': 'Animal', '/m/068hy': 

In [ ]:
#removendo as labels
df = df_expandido[df_expandido['positive_labels'].isin(labels_queridas)]

#criando uma coluna com as descrições
df['labels'] = df['positive_labels'].replace(labels_queridas)

df
#print(len(df))
#rint(len(df['positive_labels'].unique()))
#print(df['positive_labels'].value_counts())

<ipython-input-11-24615339f22e>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['labels'] = df['positive_labels'].replace(labels_queridas)


,YTID,start_seconds,end_seconds,positive_labels,labels
0,--PJHxphWEs,30.0,40.0,/m/09x0r,Speech
3,--aE2O5G5WE,0.0,10.0,/m/03fwl,Goat
4,--aE2O5G5WE,0.0,10.0,/m/04rlf,Music
5,--aE2O5G5WE,0.0,10.0,/m/09x0r,Speech
8,--aaILOrkII,200.0,210.0,/m/032s66,gunfire
...,...,...,...,...,...
52873,zz8TGV83nkE,80.0,90.0,/m/02mk9,Engine
52875,zz8TGV83nkE,80.0,90.0,/m/07yv9,Vehicle
52876,zzlK8KDqlr0,370.0,380.0,/m/01m2v,Computer keyboard
52878,zzlK8KDqlr0,370.0,380.0,/m/09x0r,Speech


In [ ]:
#em 'positive_labels", padronizar as labels todas com letra minúscula e _ no lugar do espaço

df['labels'] = df['labels'].str.lower().str.replace(' ', '_')
print(df.head())

          YTID  start_seconds  end_seconds positive_labels   labels
0  --PJHxphWEs           30.0         40.0        /m/09x0r   speech
3  --aE2O5G5WE            0.0         10.0        /m/03fwl     goat
4  --aE2O5G5WE            0.0         10.0        /m/04rlf    music
5  --aE2O5G5WE            0.0         10.0        /m/09x0r   speech
8  --aaILOrkII          200.0        210.0       /m/032s66  gunfire


<ipython-input-12-9521fea2e283>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['labels'] = df['labels'].str.lower().str.replace(' ', '_')


Parte chata de começar a ver os labels que serão descartados do banco de dados

In [ ]:
# Contar a frequência de cada label_description
label_counts = df['labels'].value_counts()

labels_to_remove = label_counts[label_counts < 100].index.tolist()
labels_to_keep = ['police_car_(siren)','fire_truck_(siren)','church_bell','civil_defense_siren']

#Remover os labels que devem ser mantidos da lista de remoção
labels_to_remove = [label for label in labels_to_remove if label not in labels_to_keep]
df_filtrado = df[~df['labels'].isin(labels_to_remove)]

print("A remover:", len(labels_to_remove))
print(labels_to_remove)

# Imprimir o tamanho do DataFrame após a remoção
print('Tamanho do df:', len(df_filtrado))
print(len(df_filtrado['labels'].unique()))

A remover: 173
['hiss', 'drum_kit', 'snare_drum', 'piano', 'faucet', 'rimshot', 'bass_drum', 'xylophone', 'rooster', 'horse', 'clickety-clack', 'power_tool', 'truck', 'goose', 'thunder', 'hammond_organ', 'rock_music', 'ambulance_(siren)', 'ocean', 'cello', 'sink_(filling_or_washing)', 'glockenspiel', 'raindrop', 'drum_roll', 'cheering', 'thunderstorm', 'trombone', 'trumpet', 'surf', 'applause', 'fiddle', 'cymbal', 'motor_vehicle_(road)', 'cricket', 'tabla', 'pop_music', 'explosion', 'train_horn', 'saxophone', 'distortion', 'waterfall', 'electric_piano', 'frog', 'environmental_noise', 'fire', 'zither', 'flute', 'alarm_clock', 'slide_guitar', 'steam', 'bovinae', 'airplane', 'whistling', 'machine_gun', 'boom', 'artillery_fire', 'mallet_percussion', 'french_horn', 'breathing', 'double_bass', 'coo', 'door', 'harpsichord', 'hip_hop_music', 'turkey', 'electronic_organ', 'car_passing_by', 'television', 'fusillade', 'belly_laugh', 'happy_music', 'telephone_bell_ringing', 'baby_laughter', 'pour'

In [ ]:
# Contar a frequência de cada label_description
label_counts1 = df_filtrado['labels'].value_counts()

labels_to_remove1 = label_counts1[((label_counts1 > 100) & (label_counts1 < 7000))].index.tolist()
labels_to_keep1 = ['speech']

#Remover os labels que devem ser mantidos da lista de remoção
labels_to_remove1 = [label for label in labels_to_remove1 if label not in labels_to_keep1]
df_filtrado1 = df_filtrado[~df_filtrado['labels'].isin(labels_to_remove1)]

print("A remover:", len(labels_to_remove1))
print(labels_to_remove1)

#Imprimir o tamanho do DataFrame após a remoção
print('Tamanho do df:', len(df_filtrado1))
print(len(df_filtrado1['labels'].unique()))
print(df_filtrado1['labels'].unique())

A remover: 52
['music', 'vehicle', 'small_room', 'animal', 'musical_instrument', 'singing', 'pets', 'guitar', 'plucked_string_instrument', 'car', 'water', 'dog', 'percussion', 'woodwind_instrument', 'urban_or_manmade', 'rural_or_natural', 'water_vehicle', 'brass_instrument', 'siren', 'fowl', 'drum', 'rail_transport', 'train', 'engine', 'train_wagon', 'gunfire', 'bird', 'insect', 'large_room_or_hall', 'wind', 'wood', 'kid_speaking', 'crowd', 'rub', 'wind_noise_(microphone)', 'keyboard_(musical)', 'emergency_vehicle', 'pizzicato', 'bird_call', 'farm_animals', 'cat', 'organ', 'mechanisms', 'housefly', 'bowed_string_instrument', 'rain', 'laughter', 'electronic_music', 'aircraft', 'effects_unit', 'hum', 'tools']
Tamanho do df: 5982
5
['speech' 'church_bell' 'police_car_(siren)' 'civil_defense_siren'
 'fire_truck_(siren)']


Usar um dicionário para unir categorias já conhecidas

In [ ]:
label_mapping = {
    'dogs': 'dog',
    'bark': 'dog',
    'boom': 'explosion',
    'ding-dong': 'doorbell',
    'telephone_bell_ringing': 'telephone',
    'yell': 'screaming',
    'meow': 'cat',
    'bang': 'shotguns',
    'tubular_bells': 'bell',
    'church_bell': 'bell',
    'fusillade': 'shotgun',
    'bang': 'shotgun',
    'gunfire': 'shotgun',
    'firecracker': 'fireworks',
    'computer_keyboard': 'keyboard_typing',
    'raindrop': 'pouring_water',
    'oink': 'pig',
    'pour': 'pouring_water',
    'belly_laugh': 'laughing',
    'sawing': 'hand_saw',
    'engine_knocking': 'engine',
    'knock': 'door_wood_knock',
    'applause': 'clapping',
    'insect': 'insects',
    'honk': 'car_horn',
    'bird': 'chirping_birds',
    'aircraft': 'airplane',
    'laughter': 'laughing',
    'cricket': 'crickets',
    'machine_gun': 'shotgun',
    'aircraft_engine': 'airplane',
    'infant_cry': 'crying_baby',
    'alarm_clock': 'clock_alarm'
}

# Substituindo
df_filtrado1['labels'] = df_filtrado1['labels'].replace(label_mapping)

print(len(df_filtrado1['labels'].unique()))
print(df_filtrado1['labels'].unique())

5
['speech' 'bell' 'police_car_(siren)' 'civil_defense_siren'
 'fire_truck_(siren)']


<ipython-input-15-e5cfe7e8a10d>:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado1['labels'] = df_filtrado1['labels'].replace(label_mapping)


In [ ]:
pd.set_option('display.max_rows', None)
df_filtrado1['labels'].value_counts()

labels
speech                 5735
bell                     66
police_car_(siren)       61
civil_defense_siren      60
fire_truck_(siren)       60
Name: count, dtype: int64

In [ ]:
df_filtrado2 = df_filtrado1.copy()
df_filtrado2 = df_filtrado1[df_filtrado1['labels'] != 'speech']
df_filtrado2.reset_index(drop=True)

,YTID,start_seconds,end_seconds,positive_labels,labels
0,-FT5MbWPCzg,540.0,550.0,/m/03w41f,bell
1,-_dElQcyJnA,30.0,40.0,/m/04qvtq,police_car_(siren)
2,-gKNRXbpAKs,30.0,40.0,/m/0dgbq,civil_defense_siren
3,-z47zN0YyE4,420.0,430.0,/m/012ndj,fire_truck_(siren)
4,03NLMEMi8-I,30.0,40.0,/m/0dgbq,civil_defense_siren
5,0JKcTVpby0I,30.0,40.0,/m/0dgbq,civil_defense_siren
6,0JPT13OUVV8,220.0,230.0,/m/012ndj,fire_truck_(siren)
7,0PhU-PIsUMw,80.0,90.0,/m/0dgbq,civil_defense_siren
8,2oT1tf8FRoU,260.0,270.0,/m/0dgbq,civil_defense_siren
9,3uZy0wkterE,30.0,40.0,/m/012ndj,fire_truck_(siren)


In [ ]:
#dropar 3000 linhas aleatórias da labels speach

import pandas as pd
import numpy as np

# Selecionar linhas aleatórias
df_filtrado1 = df_filtrado1.sample(frac=1).reset_index(drop=True)

# Selecionar linhas que sejam da label "speech"
df_filtrado1 = df_filtrado1[df_filtrado1['labels'] == 'speech']

# Selecionar 3000 linhas aleatórias
df_filtrado1 = df_filtrado1.sample(n=700)

# Imprimir o tamanho do DataFrame após a remoção
print('Tamanho do df:', len(df_filtrado1))
print(df_filtrado1)


Tamanho do df: 700
             YTID  start_seconds  end_seconds positive_labels  labels
4022  RK337pbPmkQ           40.0         50.0        /m/09x0r  speech
5680  2E0OsrYbzAI           10.0         20.0        /m/09x0r  speech
4435  xb57zcLuz9U          280.0        290.0        /m/09x0r  speech
2738  dAykqVD5n0U           20.0         30.0        /m/09x0r  speech
5572  BQikVnHehaM           30.0         40.0        /m/09x0r  speech
2742  vmAsxFQbEg4          190.0        200.0        /m/09x0r  speech
3127  uw0Piewnkmw           70.0         80.0        /m/09x0r  speech
1270  kP1ysCvMSWI           10.0         20.0        /m/09x0r  speech
2068  w5JSqUrG96Q          270.0        280.0        /m/09x0r  speech
5279  G_hxZ38_4Rw            0.0         10.0        /m/09x0r  speech
2826  s_FRrNNlP_I           70.0         80.0        /m/09x0r  speech
5731  CU1euq2Zl5E          520.0        530.0        /m/09x0r  speech
5035  HVv0jG7J0to          490.0        500.0        /m/09x0r  speech
5

Consultar áudios

In [ ]:
consulta = df_filtrado1[df_filtrado1['labels'].isin(['door_wood_knock'])]
len(consulta)

0

In [ ]:
#Concatenar df_filtrado1 e df_filtrado2

df_final = pd.concat([df_filtrado1, df_filtrado2], axis=0)
df_final.reset_index(drop=True)

,YTID,start_seconds,end_seconds,positive_labels,labels
0,RK337pbPmkQ,40.0,50.0,/m/09x0r,speech
1,2E0OsrYbzAI,10.0,20.0,/m/09x0r,speech
2,xb57zcLuz9U,280.0,290.0,/m/09x0r,speech
3,dAykqVD5n0U,20.0,30.0,/m/09x0r,speech
4,BQikVnHehaM,30.0,40.0,/m/09x0r,speech
5,vmAsxFQbEg4,190.0,200.0,/m/09x0r,speech
6,uw0Piewnkmw,70.0,80.0,/m/09x0r,speech
7,kP1ysCvMSWI,10.0,20.0,/m/09x0r,speech
8,w5JSqUrG96Q,270.0,280.0,/m/09x0r,speech
9,G_hxZ38_4Rw,0.0,10.0,/m/09x0r,speech


In [ ]:
pd.set_option('display.max_rows', 20)
df_filtrado1.reset_index(drop=True)

,YTID,start_seconds,end_seconds,positive_labels,labels
0,RK337pbPmkQ,40.0,50.0,/m/09x0r,speech
1,2E0OsrYbzAI,10.0,20.0,/m/09x0r,speech
2,xb57zcLuz9U,280.0,290.0,/m/09x0r,speech
3,dAykqVD5n0U,20.0,30.0,/m/09x0r,speech
4,BQikVnHehaM,30.0,40.0,/m/09x0r,speech
...,...,...,...,...,...
695,hyBnFl57dwA,30.0,40.0,/m/09x0r,speech
696,wjxD3Ix5TaI,200.0,210.0,/m/09x0r,speech
697,pr1NVYudVis,40.0,50.0,/m/09x0r,speech
698,I2FnnsvMj6Y,280.0,290.0,/m/09x0r,speech


In [ ]:
list = ['footsetps','clock_alarm','crying_baby','chainsaw','fireworks','environmental_noise','car_alarm','telephone','doorbell','screaming','fire',
        'fire_alarm','door_wood_knock']
df_adicional = df_filtrado1[df_filtrado1['labels'].isin(list)]
len(df_adicional)

0

##**1.3 -** Extração dos aúdios .wav

In [ ]:
!pip install pytube moviepy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 2.4 MB/s eta 0:00:00


In [ ]:
from pytube import YouTube
from moviepy.editor import *
from moviepy.editor import VideoFileClip
import os

# Diretório onde os arquivos de áudio serão salvos
output_dir = '/content/audios_avaliacao2'
os.makedirs(output_dir, exist_ok=True)

def download_and_extract_audio(row):
    yt_id = row['YTID']
    start_seconds = row['start_seconds']
    end_seconds = row['end_seconds']
    label = row['positive_labels']
    descrip = row['labels']

    # URL do vídeo no YouTube
    url = f'https://www.youtube.com/watch?v={yt_id}'

    try:
        # Baixar o vídeo
        yt = YouTube(url)
        stream = yt.streams.filter(only_audio=True).first()
        downloaded_file = stream.download()

        # Extrair áudio
        audio_clip = AudioFileClip(downloaded_file)
        audio_clip = audio_clip.subclip(start_seconds, end_seconds)

        # Nome do arquivo de saída
        output_filename = os.path.join(output_dir, f'{yt_id}_{descrip}.wav')

        # Salvar o arquivo de áudio
        audio_clip.write_audiofile(output_filename, codec='pcm_s16le')

        # Fechar os clipes para liberar memória
        audio_clip.close()

        # Remover o arquivo de vídeo baixado
        os.remove(downloaded_file)

    except Exception as e:
        # Captura exceções e continua com o próximo vídeo
        if 'HTTP Error 429: Too Many Requests' in str(e):
            print(f'Too many requests for video {yt_id}: {e}')

In [ ]:
#Função que realizará pausas durante as solicitações
import time
import random

num_requests = 0
total_requests = 0
total_videos = len(df_filtrado2)

def download_audio_with_pause(df,num_requests,total_requests,total_videos):

  for index, row in df.iterrows():
    download_and_extract_audio(row)

    num_requests += 1
    total_requests += 1

    # Checar se 150 dowloads foram feitos
    if num_requests == 300:
      print(f"Downloaded {total_requests} videos out of {total_videos}")

      # Pausa de 5 min
      print(f"Downloaded {num_requests}")
      print("Pausing for 5 minutes")
      time.sleep(200)
      num_requests = 0

      # Checar se todos os dowloads foram feitos
      if total_requests == total_videos:
        break

In [ ]:
download_audio_with_pause(df_filtrado2,num_requests,total_requests,total_videos)

MoviePy - Writing audio in /content/audios_avaliacao2/-FT5MbWPCzg_bell.wav


MoviePy - Done.


MoviePy - Writing audio in /content/audios_avaliacao2/-_dElQcyJnA_police_car_(siren).wav


MoviePy - Done.


MoviePy - Writing audio in /content/audios_avaliacao2/-gKNRXbpAKs_civil_defense_siren.wav


...

MoviePy - Writing audio in /content/audios_avaliacao2/y_ffDQjGh7Y_civil_defense_siren.wav


MoviePy - Done.


MoviePy - Writing audio in /content/audios_avaliacao2/yp4YBTyX7gk_bell.wav


MoviePy - Done.
MoviePy - Writing audio in /content/audios_avaliacao2/ysNK5RVF3Zw_fire_truck_(siren).wav


MoviePy - Done.


In [ ]:
# Caminho para a pasta contendo os áudios
caminho_pasta = '/content/audios_adicionais'

# Comando para contar o número de arquivos de áudio na pasta
!ls -1 $caminho_pasta | wc -l


993


In [ ]:
# Import necessary libraries
from google.colab import drive

# Define the source path
source_path = '/content/audios_avaliacao2'

# Define the destination path
destination_path = '/content/drive/My Drive/audios_avaliacao2'

# Mount Google Drive
drive.mount('/content/drive')

# Change directory to the source folder
%cd /content

# Zip the folder
!zip -r audios_avaliacao2.zip audios_avaliacao2

# Copy the zipped file to Google Drive
!cp audios_avaliacao2.zip /content/drive/My\ Drive


Mounted at /content/drive
/content
  adding: audios_avaliacao2/ (stored 0%)
  adding: audios_avaliacao2/_Tq-0qgB7m4_civil_defense_siren.wav (deflated 40%)
  adding: audios_avaliacao2/7Hcizjr7zB8_police_car_(siren).wav (deflated 7%)
  adding: audios_avaliacao2/_JTHtcRHxnw_bell.wav (deflated 34%)
  adding: audios_avaliacao2/5otuWD5-Zhc_bell.wav (deflated 10%)
  adding: audios_avaliacao2/_lkuZpt1Jt0_police_car_(siren).wav (deflated 4%)
  adding: audios_avaliacao2/pMDfOwaHg6A_bell.wav (deflated 29%)
  adding: audios_avaliacao2/ZCgE23s5bt0_fire_truck_(siren).wav (deflated 7%)
  adding: audios_avaliacao2/8DaEd5KbnnA_police_car_(siren).wav (deflated 4%)
  adding: audios_avaliacao2/j1cL9eMKgck_fire_truck_(siren).wav (deflated 1%)
  adding: audios_avaliacao2/lvtAPn1aqQc_civil_defense_siren.wav (deflated 6%)
  adding: audios_avaliacao2/E7tueVnDYDE_police_car_(siren).wav (deflated 2%)
  adding: audios_avaliacao2/y1USVCTNyVI_civil_defense_siren.wav (deflated 20%)
  adding: audios_avaliacao2/NuZqqc

In [ ]:
# Importar a biblioteca necessária
from google.colab import drive
import os

# Montar o Google Drive
drive.mount('/content/drive')

# Definir o caminho da pasta de origem e destino
source_path = '/content/audios_adicionais'
zip_path = '/content/audios_adicionais.zip'
drive_path = '/content/drive/My Drive/audios_adicionais.zip'

# Compactar a pasta
os.system(f'zip -r {zip_path} {source_path}')

# Verificar se o arquivo ZIP foi criado
if os.path.exists(zip_path):
    print("Arquivo ZIP criado com sucesso.")

    # Listar os arquivos no diretório para verificar a criação do arquivo ZIP
    os.system('ls -lh /content')

    # Verificar o conteúdo do arquivo ZIP
    os.system(f'unzip -l {zip_path}')

    # Copiar o arquivo compactado para o Google Drive
    os.system(f'cp {zip_path} "{drive_path}"')

    # Mensagem de sucesso
    print("Arquivo ZIP foi salvo no Google Drive com sucesso.")
else:
    print("Falha na criação do arquivo ZIP.")



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Arquivo ZIP criado com sucesso.
Arquivo ZIP foi salvo no Google Drive com sucesso.


In [ ]:
# listar quantos arquivos tem na minha pasta /content/audios_audioset

!ls -1 /content/audios_audioset | wc -l


4887


In [ ]:
print(df_filtrado1.iloc[1])

YTID               --cB2ZVjpnA
start_seconds             30.0
end_seconds               40.0
positive_labels      /m/01y3hg
labels             smoke_alarm
Name: 10, dtype: object
